In [93]:
from Bio import SeqIO
import subprocess
from os.path import expanduser

##IF blastDB does not yet exist, make one using:
wd=expanduser("~/Mosquitoes/ResistanceGenes/") #Change the directory as appropriate
gffFile=wd+"/InputData/GCF_002204515.2_AaegL5.0_genomic.gff" #Get these from NCBI/ENA
fastaFile=wd+"/InputData/GCF_002204515.2_AaegL5.0_genomic.fna"  #Get these from NCBI/ENA


blastCommand=f'makeblastdb -in {fastaFile} -title AaegL5 -out {wd}/blastDB/AaegL5 -dbtype nucl  -blastdb_version 4 '
subprocess.run(blastCommand, shell=True, executable="/bin/bash", stdout=subprocess.PIPE, stderr=subprocess.PIPE)



chrLength={}
exons={}
with open(gffFile) as file:
    for line in file:
        if line[0:2]=="NC":
            #these are the chromosomes, unplaced contigs start with NW
            values=line.split("\t")
            if values[2]=="region":
               chrLength[values[0]]=int(values[4]) 

targetmRNA={}
targetGenePositions={}
for chr in chrLength:
    targetGenePositions[chr]=[]
    targetmRNA[chr]=[]
    for i in range(1,6):
        targetGenePositions[chr].append( int(chrLength[chr] / 6 * i) )


def GetBlastMatchesLength(fastaSequence):
    with open(f'{wd}/temp.fasta', "w") as outputFasta:  
        outputFasta.write(">ExonSeqeuence\n"+fastaSequence+"\n")
    blastCommand="blastn -query "+wd+"/temp.fasta -task 'megablast'  -max_target_seqs 1000 -db "+wd+"/blastDB/AaegL5 -num_threads 8 -evalue 0.01 -word_size 28 -outfmt '6 delim=  qseqid qstart qend sseqid sstart send pident evalue'"
    gff=subprocess.run(blastCommand, shell=True, executable="/bin/bash", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    result=0
    for line in gff.stdout.decode().split("\n"):
        if len(line)>0:
            result+=int(line.split("\t")[2])-int(line.split("\t")[1])
    return result

def GetAnnotationValue(targetAnnotationLabel, wholeAnnotationField):
    annotations=wholeAnnotationField.strip().split(";")
    for annotation in annotations:
        annLabel,annValue=annotation.split("=")
        if annLabel==targetAnnotationLabel:
            return annValue
    return "AnnotationNotFound"

def CollectExonSequences(chromosome, mRNAid):
    targetExons=[] # [(exon1 start, exon1 end), (exon2 start, exon2 end)]
    with open(gffFile) as file:
        for line in file:
            if line[0]!="#":
                values=line.split("\t")
                if values[0]==chromosome and values[2]=="exon":
                    transcriptID=GetAnnotationValue("transcript_id", values[8].strip())
                    if mRNAid==transcriptID:
                        targetExons.append( (int(values[3]), int(values[4])) )
    exonSequence=""
    for chr in SeqIO.parse(fastaFile, "fasta"):
        if chr.id in chromosome:
            for exonStart, exonEnd in targetExons:
                exonSequence=exonSequence+str(chr.seq[exonStart:exonEnd])
    return exonSequence

print(targetGenePositions)
with open(wd+"resistanceGeneExonSequences.fasta","w") as exonOutputFile:
    with open(gffFile) as file:
        for line in file:
            if line[0]!="#":
                values=line.split("\t")
                if values[2]=="mRNA" and values[0] in targetGenePositions and len(targetGenePositions[values[0]])>0:
                    if int(values[4])-int(values[3])>5000 and int(values[4])-int(values[3])<25000:
                        #check if gene coordinates are just after the first target chromosomes position
                        #add mRNA and remove the target position
                        if int(values[3])>targetGenePositions[values[0]][0]:
                            #get transcriptid:
                            transcriptID=GetAnnotationValue("transcript_id", values[8].strip())
                            trascriptExonsSequence=CollectExonSequences(values[0],transcriptID)
                            blastTotalLength=GetBlastMatchesLength(trascriptExonsSequence)
                            if abs(blastTotalLength-len(trascriptExonsSequence)) < 10:
                                targetmRNA[values[0]].append(line)
                                targetGenePositions[values[0]].pop(0)
                                exonOutputFile.write(">"+transcriptID+"\n"+trascriptExonsSequence+"\n")
                                print(values[0])
                                print(len(targetGenePositions[values[0]]))




{'NC_035107.1': [51804503, 103609007, 155413511, 207218014, 259022518], 'NC_035108.1': [79070952, 158141905, 237212858, 316283810, 395354763], 'NC_035109.1': [68296278, 136592556, 204888835, 273185113, 341481391], 'NC_035159.1': [2798, 5596, 8395, 11193, 13991]}
NC_035107.1
4
NC_035107.1
3
NC_035107.1
2
NC_035107.1
1
NC_035107.1
0
NC_035108.1
4
NC_035108.1
3
NC_035108.1
2
NC_035108.1
1
NC_035108.1
0
NC_035109.1
4
NC_035109.1
3
NC_035109.1
2
NC_035109.1
1
NC_035109.1
0


In [94]:
promoterBuffer=300

resitanceGenes=[("XM_021851332.1",161486025,161645623,"ACE","NC_035109.1"),
                ("XM_021840622.1", 41628786,41861946,"GABA","NC_035108.1"),
                ("XM_021852340.1",315926360,316405638,"VSSC","NC_035109.1"),
                ("XM_021846286.1",351633367,351634957,"GSTE2","NC_035108.1"),
                ("YP_009389261.1",1298,2834,"COX1","NC_035159.1")]


def write_to_bed(line, file_handle):
        values=line.split("\t")
        values[3]=str(int(values[3])-promoterBuffer)
        values[4]=str(int(values[4])-promoterBuffer)
        file_handle.write( "\t".join( [values[0], values[3], values[4],GetAnnotationValue("transcript_id", values[8].strip())] ) +"\n")

with open(f'{wd}/InputData/mappedRegions.bed', "w") as bed_output:        
    for chr, target in targetmRNA.items():
        for item in targetmRNA[chr]:
             write_to_bed(item, bed_output)
    for value in resitanceGenes:
         bed_output.write("\t".join(  [value[4], str(value[1]-promoterBuffer), str(value[2]+promoterBuffer), value[0] ]  )+"\n")

subprocess.run("bedtools getfasta -name  -fi "+fastaFile+" -bed "+wd+"/InputData/mappedRegions.bed > "+wd+"/InputData/mappedRegions.fasta ", shell=True, executable="/bin/bash")

CompletedProcess(args='bedtools getfasta -name  -fi ~/Mosquitoes/GCF_002204515.2_AaegL5.0_genomic.fna -bed ~/Mosquitoes/temp.bed > ~/Mosquitoes/ResistanceGenes/genesV2.fasta ', returncode=0)